In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import ensemble,metrics,cross_validation

C:\Users\sharm_000\Anaconda2\envs\py36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pd.read_csv('train.csv')
test_=pd.read_csv('test.csv')

In [3]:
data['val']='train'
test_['val']='test'

In [4]:
all_data=pd.concat([data,test_])

In [5]:
cat_cols=[i for i in all_data.columns if 'cat' in i]

In [6]:
new_data=pd.get_dummies(all_data,columns=cat_cols,drop_first=True)

In [7]:
new_data.shape

(1488028, 216)

In [8]:
all_data.columns

Index(['id', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11', 'ps_car_11_cat', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'target', '

In [13]:
X=all_data.columns[1:-2]
Y='target'

In [14]:
train_data=all_data[all_data['val']=='train']
test_data=all_data[all_data['val']=='test']

In [15]:
def gini(list_of_values):
    sorted_list = sorted(list(list_of_values))
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2
    return (fair_area - area) / fair_area
  
def normalized_gini(y_pred, y):
    normalized_gini = gini(y_pred)/gini(y)
    return normalized_gini

In [16]:
train_X,test_X,train_y,test_y=cross_validation.train_test_split(train_data[X],train_data[Y],test_size=.3,stratify=train_data[Y])
train_X.shape,train_y.shape,test_X.shape,test_y.shape

((416648, 57), (416648,), (178564, 57), (178564,))

In [17]:
model=xgb.XGBClassifier().fit(train_X,train_y)

In [18]:
pred=model.predict(test_X)

In [21]:
pred_new=model.predict_proba(test_X)

In [38]:
normalized_gini(pred_new[:,1],test_y)

0.24292677865704895

In [23]:
new_pred=model.predict_proba(test_data[X])

In [24]:
new_pred[:,1]

array([ 0.02682137,  0.0293278 ,  0.03193527, ...,  0.04216085,
        0.0272694 ,  0.02627678], dtype=float32)

In [30]:
sub=pd.DataFrame(test_['id'])

In [31]:
sub['target']=new_pred[:,1]

In [33]:
sub.to_csv('sub.csv',index=False)